In [1]:
# Init chroma db with city data from md files
# Augment llama 2 with city data from chroma db
# Suggest cities to visit based on user input

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from chroma import init_chroma

In [9]:
db = init_chroma()
llm = Ollama()

Split 10 documents into 28 chunks.
Phra Kaew, and experience the unique charm of floating markets. The city's street food is renowned, offering a variety of flavorful dishes at affordable prices. Bangkok is also a shopping paradise, with everything from high-end malls to local markets offering a range of goods.
{'source': 'data/cities/bangkok.md', 'start_index': 195}
Saved 28 chunks to data/chroma.


In [5]:
PROMPT_TEMPLATE = """
<<SYS>>
You are a travel agent. You have a database of cities and their attractions. You are tasked with suggesting cities to visit based on user input.
<</SYS>>

[INST]
Tell the user about the city they ask about. If it does not exist in the context, tell them sorry, you don't have that information.

context: {context}
[/INST]

User: Tell me about the city {city}
"""

In [38]:
query_text = input("Enter a city: ")

results = db.similarity_search_with_relevance_scores(query_text, k=1)
context = "Unable to find matching city"

if len(results) > 0:
  [document, score] = results[0]
  print(document)
  print(score)

  if(score > 0.5):
    context = document.page_content

prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
chain = prompt | llm

chain.invoke({ "context": document.page_content, "city": query_text})

/Users/gustaf/opt/miniconda3/lib/python3.8/site-packages/langchain_core/vectorstores.py:331: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="and Topkapi Palace, witnessing the city's rich history firsthand. The Grand Bazaar and Spice Bazaar offer a vibrant shopping experience, where visitors can find traditional Turkish crafts, spices, and textiles. Istanbul's culinary scene is also a major draw, with a wide array of delicious Turkish", metadata={'source': 'data/cities/istanbul.md', 'start_index': 197}), -0.03805097048394135)]
  warnings.warn(


page_content="and Topkapi Palace, witnessing the city's rich history firsthand. The Grand Bazaar and Spice Bazaar offer a vibrant shopping experience, where visitors can find traditional Turkish crafts, spices, and textiles. Istanbul's culinary scene is also a major draw, with a wide array of delicious Turkish" metadata={'source': 'data/cities/istanbul.md', 'start_index': 197}
-0.03805097048394135


"Travel Agent: Of course! Istanbul is a beautiful and vibrant city that offers a unique blend of Eastern and Western cultures. It is located in Turkey and is known for its rich history, stunning architecture, and delicious cuisine.\n\nSome popular attractions in Istanbul include the Topkapi Palace, which was the primary residence of the Ottoman sultans for over 400 years and now serves as a museum showcasing the city's rich history. The Grand Bazaar and Spice Bazaar are also must-visit places, offering a vibrant shopping experience where visitors can find traditional Turkish crafts, spices, and textiles.\n\nIstanbul's culinary scene is also a major draw, with a wide array of delicious Turkish dishes such as kebabs, baklava, and Turkish delight. The city is also home to many world-class museums, including the Istanbul Museum of Modern Art and the Sakıp Sabancı Museum.\n\nIn addition to its cultural and historical attractions, Istanbul is also known for its vibrant nightlife, with many b